In [ ]:
# !python3 -m pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
# !python3 -m pip install simpletransformers
# !python3 -m pip install pandas scikit-learn
# !python3 -m pip install transformers-interpret

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from simpletransformers.classification import MultiLabelClassificationModel
import ast
EVALMODE=1

In [2]:
df_train = pd.read_csv("../data/classification/train/train_complex_aug.tsv",sep="\t")
df_train.drop("labels",axis=1,inplace=True)
df_train= df_train.rename(columns={'multi':"labels"})
df_train = df_train[df_train["labels"] != "[1,1,1]"]
df_train["labels"] = [ast.literal_eval(i) for i in df_train.labels.values.tolist()] 
# print(set(df_train["labels"].values.tolist()))
df_train


,text,labels
0,movie vara level la Erika poguthu,"[0, 0, 0]"
1,movie beethi level la pole mairu,"[0, 0, 1]"
2,movie .... sanghu kulla mokka poguthu,"[0, 1, 0]"
3,movie vara da asinga Erika fucking,"[0, 1, 1]"
4,Padam nalla comedy padama irukum polaye..,"[0, 0, 0]"
...,...,...
109955,Just WTF WORST Bangam jaldra shit,"[0, 1, 1]"
109956,நாமக்கல் மாவட்டம் வன்னியர் சார்பாக திரௌபதி பட...,"[0, 0, 0]"
109957,punda மாவட்டம் வன்னியர் pooolu shit mokka போடா...,"[1, 1, 1]"
109958,நாமக்கல் மாவட்டம் வன்னியர் Dai திரௌபதி flop e ...,"[0, 1, 1]"


In [3]:
df_test = pd.read_csv("../data/classification/test/test_tamil_offensive_binary.tsv",sep="\t")
df_test["labels"] = [[i,i,i] for i in df_test["labels"]]
df_test

,text,labels
0,14.12.2018 epo trailer pathutu irken ... Semay...,"[0, 0, 0]"
1,Paka thana poro movie la Enna irukunu,"[0, 0, 0]"
2,Suriya anna vera level anna mass,"[0, 0, 0]"
3,suma kaththaatha da sound over a pooda kudaath...,"[1, 1, 1]"
4,ஆனவக் கொலைகள் நடக்காவிடில் ...நம் அடையாளம் அளி...,"[1, 1, 1]"
...,...,...
4227,Vijay going to next level thalaivaaaaa,"[0, 0, 0]"
4228,மண்ணு பொண்ணு ரெண்டுமே ஒன்னு அதுல எவன் கைய வச்ச...,"[1, 1, 1]"
4229,asuran= aadukalam+pudupettai+ wada chennai..ye...,"[0, 0, 0]"
4230,Vijay's all movies look like same.,"[0, 0, 0]"


# Classification Training (New labels)

In [4]:
# train_using_new_label

In [5]:
if EVALMODE==0:
    model_args = {
        "num_train_epochs":5,
        "overwrite_output_dir":True, 
        "manual_seed":42,
        "max_seq_length":150,
        #sliding_window=True,
        "train_batch_size":64,
        "eval_batch_size":64,
        "best_model_dir":"models/best_xlmbert_complex_aug_multi",
        "output_dir":"models/outputsxlm_complex",
        "do_lower_case":True,
        "early_stopping_patience":2,
        "save_model_every_epoch":True,
        "early_stopping_consider_epochs":True,
        "use_early_stopping":False,
        "evaluate_during_training":True,
        #wandb_project='ST-Hope-CELoss',
        "learning_rate":3e-05}
    
    model = MultiLabelClassificationModel(
        model_type='xlmroberta',
        model_name='xlm-roberta-base', 
        use_cuda=torch.cuda.is_available(),
        cuda_device="0",
        num_labels=3, 
        args=model_args)
    
#roberta roberta-base
#xlmroberta xlm-roberta-base
#bert bert-multilingual-base-uncased

In [6]:

if EVALMODE==0:
    model.train_model(df_train,eval_df=df_test)
elif EVALMODE==1:
    model_args = ClassificationArgs(
        eval_batch_size=64)
    model = MultiLabelClassificationModel(
        model_type='xlmroberta',
        model_name='models/outputsxlm_complex/checkpoint-3438-epoch-2', 
        use_cuda=torch.cuda.is_available(),
        cuda_device="0", args=model_args)

In [7]:
result, model_outputs, wrong_predictions = model.eval_model(df_test)
print(result)


  0%|          | 0/4232 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/67 [00:00<?, ?it/s]

{'LRAP': 1.0, 'eval_loss': 0.39737749366617914}


In [8]:
testx = [i for i in df_test["text"].values.tolist()]
testx
predictions, raw_outputs = model.predict(testx)

ypr=[]
ypr_full=[]
for i in predictions:
        ypr.append(max(i))
        ypr_full.append(i)
ygt=[]
for i in df_test["labels"].values:
        ygt.append(max(i))

print(ypr)
print(classification_report(ygt,ypr,digits=5))

np.save("xlm_roberta_ypr_complex_aug_multi.npy",np.array(ypr_full))


  0%|          | 0/4232 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [ ]:
# Interpret Predictions

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer,MultiLabelClassificationExplainer
import pandas as pd
import os
from sklearn.metrics import classification_report
import numpy as np
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
df_test



In [ ]:
model_name = "models/outputsxlm_complex/checkpoint-3438-epoch-2/"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model     = model.to('cuda:0')

testx = [i for i in df_test["text"].values.tolist()]
cls_explainer = MultiLabelClassificationExplainer(
        model,
        tokenizer)

In [ ]:
count=0
attribution_list_complex=[]

for sntce in testx[0:250]:
    word_attributions = cls_explainer(sntce,internal_batch_size=1)
    attribution_list_complex.append(word_attributions)
    count+=1
    if count%10 ==0:
        cls_explainer=None
        cls_explainer = MultiLabelClassificationExplainer(
        model,
        tokenizer)
        torch.cuda.empty_cache()
        print("Completed", count, "sentences")
    word_attributions = None
    
np.save("xlm_roberta_attribution_complex_multi_0_250.npy",np.array(attribution_list_complex))

In [ ]:
count=0
attribution_list_complex=[]

for sntce in testx[250:500]:
    word_attributions = cls_explainer(sntce,internal_batch_size=1)
    attribution_list_complex.append(word_attributions)
    count+=1
    if count%10 ==0:
        cls_explainer=None
        cls_explainer = MultiLabelClassificationExplainer(
        model,
        tokenizer)
        torch.cuda.empty_cache()
        print("Completed", count, "sentences")
    word_attributions = None
    
np.save("xlm_roberta_attribution_complex_250_500.npy",np.array(attribution_list_complex))

In [ ]:
count=0
attribution_list_complex=[]

for sntce in testx[500:750]:
    word_attributions = cls_explainer(sntce,internal_batch_size=1)
    attribution_list_complex.append(word_attributions)
    count+=1
    if count%10 ==0:
        cls_explainer=None
        cls_explainer = MultiLabelClassificationExplainer(
        model,
        tokenizer)
        torch.cuda.empty_cache()
        print("Completed", count, "sentences")
    word_attributions = None
    
np.save("xlm_roberta_attribution_complex_500_750.npy",np.array(attribution_list_complex))

In [ ]:
count=0
attribution_list_complex=[]

for sntce in testx[750:]:
    word_attributions = cls_explainer(sntce,internal_batch_size=1)
    attribution_list_complex.append(word_attributions)
    count+=1
    if count%10 ==0:
        cls_explainer=None
        cls_explainer = MultiLabelClassificationExplainer(
        model,
        tokenizer)
        torch.cuda.empty_cache()
        print("Completed", count, "sentences")
    word_attributions = None
    
np.save("xlm_roberta_attribution_complex_multi_750_900.npy",np.array(attribution_list_complex))